Special Routines are available for evolving under a diagonal Coulomb operator.  This notebook describes how to use these built in routines and how they work.

In [1]:
from itertools import product
import fqe
from fqe.hamiltonians.diagonal_coulomb import DiagonalCoulomb

import numpy as np

import openfermion as of

from scipy.linalg import expm

In [2]:
#Utility function
def uncompress_tei(tei_mat, notation='chemistry'):
    """
    uncompress chemist notation integrals

    tei_tensor[i, k, j, l] = tei_mat[(i, j), (k, l)]
    [1, 1, 2, 2] = [1, 1, 2, 2] = [1, 1, 2, 2]  = [1, 1, 2, 2]
    [i, j, k, l] = [k, l, i, j] = [j, i, l, k]* = [l, k, j, i]*

    For real we also have swap of i <> j and k <> l
    [j, i, k, l] = [l, k, i, j] = [i, j, l, k] = [k, l, j, i]

    tei_mat[(i, j), (k, l)] = int dr1 int dr2 phi_i(dr1) phi_j(dr1) O(r12) phi_k(dr1) phi_l(dr1)

    Physics notation is the notation that is used in FQE.

    Args:
        tei_mat: compressed two electron integral matrix

    Returns:
        uncompressed 4-electron integral tensor. No antisymmetry.
    """
    if notation not in ['chemistry', 'physics']:
        return ValueError("notation can be [chemistry, physics]")

    norbs = int(0.5 * (np.sqrt(8 * tei_mat.shape[0] + 1) - 1))
    basis = {}
    cnt = 0
    for i, j in product(range(norbs), repeat=2):
        if i >= j:
            basis[(i, j)] = cnt
            cnt += 1

    tei_tensor = np.zeros((norbs, norbs, norbs, norbs))
    for i, j, k, l in product(range(norbs), repeat=4):
        if i >= j and k >= l:
            tei_tensor[i, j, k, l] = tei_mat[basis[(i, j)], basis[(k, l)]]
            tei_tensor[k, l, i, j] = tei_mat[basis[(i, j)], basis[(k, l)]]
            tei_tensor[j, i, l, k] = tei_mat[basis[(i, j)], basis[(k, l)]]
            tei_tensor[l, k, j, i] = tei_mat[basis[(i, j)], basis[(k, l)]]

            tei_tensor[j, i, k, l] = tei_mat[basis[(i, j)], basis[(k, l)]]
            tei_tensor[l, k, i, j] = tei_mat[basis[(i, j)], basis[(k, l)]]
            tei_tensor[i, j, l, k] = tei_mat[basis[(i, j)], basis[(k, l)]]
            tei_tensor[k, l, j, i] = tei_mat[basis[(i, j)], basis[(k, l)]]

    if notation == 'chemistry':
        return tei_tensor
    elif notation == 'physics':
        return np.asarray(tei_tensor.transpose(0, 2, 1, 3), order='C')

    return tei_tensor


The first example we will perform is diagonal Coulomb evolution on the Hartree-Fock state.  The diagonal Coulomb operator is defined as

\begin{align}
V = \sum_{\alpha, \beta \in \{\uparrow, \downarrow\}}\sum_{p,q} V_{pq,pq}n_{p,\alpha}n_{q,\beta}
\end{align}

The number of free parpameters are $\mathcal{O}(N^{2})$ where $N$ is the rank of the spatial basis. The `DiagonalCoulomb` Hamiltonian takes either a generic 4-index tensor or the $N \times N$ matrix defining $V$.  If the 4-index tensor is given the $N \times N$ matrix is constructed along with the diagonal correction.  If the goal is to just evolve under $V$ it is recommended the user input the $N \times N$ matrix directly.

All the terms in $V$ commute and thus we can evolve under $V$ exactly by counting the accumulated phase on each bitstring.


To start out let's define a Hartree-Fock wavefunction for 4-orbitals and 2-electrons $S_{z} =0$.

In [3]:
norbs = 4
tedim = norbs * (norbs + 1) // 2
if (norbs // 2) % 2 == 0:
    n_elec = norbs // 2
else:
    n_elec = (norbs // 2) + 1
sz = 0
fqe_wfn = fqe.Wavefunction([[n_elec, sz, norbs]])
fci_data = fqe_wfn.sector((n_elec, sz))
fci_graph = fci_data.get_fcigraph()
hf_wf = np.zeros((fci_data.lena(), fci_data.lenb()), dtype=np.complex128)
hf_wf[0, 0] = 1  # right most bit is zero orbital.
fqe_wfn.set_wfn(strategy='from_data',
                raw_data={(n_elec, sz): hf_wf})
fqe_wfn.print_wfn()

Sector N = 2 : S_z = 0
a'0001'b'0001' (1+0j)


Now we can define a random 2-electron operator $V$.  To define $V$ we need a $4 \times 4$ matrix.  We will generate this matrix by making a full random two-electron integral matrix and then just take the diagonal elements

In [4]:
tei_compressed = np.random.randn(tedim**2).reshape((tedim, tedim))
tei_compressed = 0.5 * (tei_compressed + tei_compressed.T)
tei_tensor = uncompress_tei(tei_compressed, notation='physics')

diagonal_coulomb = of.FermionOperator()
diagonal_coulomb_mat = np.zeros((norbs, norbs))
for i, j in product(range(norbs), repeat=2):
    diagonal_coulomb_mat[i, j] = tei_tensor[i, j, i, j]
    for sigma, tau in product(range(2), repeat=2):
        diagonal_coulomb += of.FermionOperator(
            ((2 * i + sigma, 1), (2 * i + sigma, 0), (2 * j + tau, 1),
             (2 * j + tau, 0)), coefficient=diagonal_coulomb_mat[i, j])

dc_ham = DiagonalCoulomb(diagonal_coulomb_mat)


Evolution under $V$ can be computed by looking at each bitstring, seeing if $n_{p\alpha}n_{q\beta}$ is non-zero and then phasing that string by $V_{pq}$.  For the Hartree-Fock state we can easily calculate this phase accumulation.  The alpha and beta bitstrings are "0001" and "0001". 

In [5]:
alpha_occs = [list(range(fci_graph.nalpha()))]
beta_occs = [list(range(fci_graph.nbeta()))]
occs = alpha_occs[0] + beta_occs[0]
diag_ele = 0.
for ind in occs:
    for jnd in occs:
        diag_ele += diagonal_coulomb_mat[ind, jnd]
evolved_phase = np.exp(-1j * diag_ele)
print(evolved_phase)

# evolve FQE wavefunction
evolved_hf_wfn = fqe_wfn.time_evolve(1, dc_ham)

# check they the accumulated phase is equivalent!
assert np.isclose(evolved_hf_wfn.get_coeff((n_elec, sz))[0, 0], evolved_phase)


(0.5106803510704031+0.8597706549019976j)


We can now try this out for more than 2 electrons.  Let's reinitialize a wavefunction on 6-orbitals with 4-electrons $S_{z} = 0$ to a random state.

In [6]:
norbs = 6
tedim = norbs * (norbs + 1) // 2
if (norbs // 2) % 2 == 0:
    n_elec = norbs // 2
else:
    n_elec = (norbs // 2) + 1
sz = 0
fqe_wfn = fqe.Wavefunction([[n_elec, sz, norbs]])
fqe_wfn.set_wfn(strategy='random')
inital_coeffs = fqe_wfn.get_coeff((n_elec, sz)).copy()
print("Random initial wavefunction")
fqe_wfn.print_wfn()

Random initial wavefunction
Sector N = 4 : S_z = 0
a'000011'b'000011' (0.03607268128375084+0.0036672918724366117j)
a'000011'b'000101' (-0.01003192868120724-0.0063856908171158475j)
a'000011'b'001001' (-0.07883948877392077-0.07208241598686203j)
a'000011'b'010001' (-0.03074331287340237+0.048812354027201556j)
a'000011'b'100001' (-0.05430836792812431+0.025388413532073096j)
a'000011'b'000110' (0.05383090797761347-0.05489134688788751j)
a'000011'b'001010' (-0.024203298193198382+0.03610081604718629j)
a'000011'b'010010' (0.019788882759903502+0.04995385723673918j)
a'000011'b'100010' (-0.05124042568967651-0.030384742476212864j)
a'000011'b'001100' (0.018586999308922717+0.0068103392337386884j)
a'000011'b'010100' (0.0518859457327846-0.00023079821720621902j)
a'000011'b'100100' (-0.0047164742135019306-0.07138576242282005j)
a'000011'b'011000' (0.04325650607389748+0.05289416588792588j)
a'000011'b'101000' (0.050017584345923134+0.016470039140649824j)
a'000011'b'110000' (-0.0366590855630107+0.01975951674708

We need to build our Diagoanl Coulomb operator For this bigger system.

In [7]:
tei_compressed = np.random.randn(tedim**2).reshape((tedim, tedim))
tei_compressed = 0.5 * (tei_compressed + tei_compressed.T)
tei_tensor = uncompress_tei(tei_compressed, notation='physics')

diagonal_coulomb = of.FermionOperator()
diagonal_coulomb_mat = np.zeros((norbs, norbs))
for i, j in product(range(norbs), repeat=2):
    diagonal_coulomb_mat[i, j] = tei_tensor[i, j, i, j]
    for sigma, tau in product(range(2), repeat=2):
        diagonal_coulomb += of.FermionOperator(
            ((2 * i + sigma, 1), (2 * i + sigma, 0), (2 * j + tau, 1),
             (2 * j + tau, 0)), coefficient=diagonal_coulomb_mat[i, j])

dc_ham = DiagonalCoulomb(diagonal_coulomb_mat)


Now we can convert our wavefunction to a cirq wavefunction, evolve under the diagonal_coulomb operator we constructed and then compare the outputs.

In [8]:
cirq_wfn = fqe.to_cirq(fqe_wfn).reshape((-1, 1))
final_cirq_wfn = expm(-1j * of.get_sparse_operator(diagonal_coulomb)) @ cirq_wfn
# recover a fqe wavefunction
from_cirq_wfn = fqe.from_cirq(final_cirq_wfn.flatten(), 1.0E-8)


/Users/nickrubin/anaconda3/envs/fqe-dev/lib/python3.6/site-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


In [9]:
fqe_wfn = fqe_wfn.time_evolve(1, dc_ham)
print("Evolved wavefunction")
fqe_wfn.print_wfn()

Evolved wavefunction
Sector N = 4 : S_z = 0
a'000011'b'000011' (0.006712861975898718+0.035631795474958046j)
a'000011'b'000101' (0.005099712715661809+0.010742884644933285j)
a'000011'b'001001' (0.1035054863309383+0.026422603657326224j)
a'000011'b'010001' (-0.050410553812651915+0.028045913363812798j)
a'000011'b'100001' (0.05387381674955979-0.02629795120398022j)
a'000011'b'000110' (-0.03392440640258204-0.06899247253935126j)
a'000011'b'001010' (0.02609522481239344+0.03475784522510435j)
a'000011'b'010010' (-0.0453399179882296-0.028831919300218972j)
a'000011'b'100010' (0.059503541809196446+0.002853473736736504j)
a'000011'b'001100' (0.012038871959608252+0.015713778213041325j)
a'000011'b'010100' (0.015984141771159116+0.0493630615363555j)
a'000011'b'100100' (-0.005091504437124941-0.07135999431236435j)
a'000011'b'011000' (-0.042483295661031706+0.053517171940209204j)
a'000011'b'101000' (0.008494533104480428-0.051969835870752545j)
a'000011'b'110000' (0.038186267140509045-0.016617342093970647j)
a'00

In [10]:
print("From Cirq Evolution")
from_cirq_wfn.print_wfn()
assert np.allclose(from_cirq_wfn.get_coeff((n_elec, sz)),
                   fqe_wfn.get_coeff((n_elec, sz)))
print("Wavefunctions are equivalent")

From Cirq Evolution
Sector N = 4 : S_z = 0
a'000011'b'000011' (0.006712861975898723+0.035631795474958046j)
a'000011'b'000101' (0.005099712715661836+0.010742884644933278j)
a'000011'b'001001' (0.10350548633093835+0.02642260365732602j)
a'000011'b'010001' (-0.050410553812652+0.028045913363812666j)
a'000011'b'100001' (0.053873816749559794-0.026297951203980267j)
a'000011'b'000110' (-0.03392440640258209-0.06899247253935124j)
a'000011'b'001010' (0.02609522481239351+0.03475784522510429j)
a'000011'b'010010' (-0.04533991798822964-0.02883191930021894j)
a'000011'b'100010' (0.05950354180919645+0.0028534737367364588j)
a'000011'b'001100' (0.01203887195960828+0.01571377821304131j)
a'000011'b'010100' (0.015984141771159147+0.049363061536355494j)
a'000011'b'100100' (-0.005091504437125068-0.07135999431236431j)
a'000011'b'011000' (-0.04248329566103174+0.05351717194020919j)
a'000011'b'101000' (0.00849453310448037-0.05196983587075255j)
a'000011'b'110000' (0.03818626714050903-0.016617342093970647j)
a'000101'b'

Finally, we can compare against evolving each term of $V$ individually.

In [11]:
fqe_wfn = fqe.Wavefunction([[n_elec, sz, norbs]])
fqe_wfn.set_wfn(strategy='from_data',
                raw_data={(n_elec, sz): inital_coeffs})
for term, coeff in diagonal_coulomb.terms.items():
    op = of.FermionOperator(term, coefficient=coeff)
    fqe_wfn = fqe_wfn.time_evolve(1, op)

assert np.allclose(from_cirq_wfn.get_coeff((n_elec, sz)),
               fqe_wfn.get_coeff((n_elec, sz)))
print("Individual term evolution is equivalent")

Individual term evolution is equivalent
